In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import tkinter as tk
from tkinter import filedialog, ttk
from datetime import datetime
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

class MultiAxisPlotter:
    def __init__(self, root):
        self.root = root
        self.root.title("Построение временных рядов с несколькими осями")
        self.root.geometry("1200x800")
        
        # Переменные для данных
        self.data = None
        self.columns = []
        self.time_columns = []
        self.value_columns = []
        self.colors = ['red', 'cyan', 'green', 'white', 'yellow', 'magenta']
        
        # Создание элементов интерфейса
        self.create_ui()
        
    def create_ui(self):
        # Создание фреймов
        control_frame = ttk.Frame(self.root, padding="10")
        control_frame.pack(fill=tk.X)
        
        plot_frame = ttk.Frame(self.root)
        plot_frame.pack(fill=tk.BOTH, expand=True)
        
        # Элементы управления
        ttk.Button(control_frame, text="Загрузить данные", command=self.load_data).grid(row=0, column=0, padx=5, pady=5)
        
        ttk.Label(control_frame, text="Начальная дата/время:").grid(row=0, column=1, padx=5, pady=5)
        self.start_date_entry = ttk.Entry(control_frame, width=20)
        self.start_date_entry.grid(row=0, column=2, padx=5, pady=5)
        
        ttk.Label(control_frame, text="Конечная дата/время:").grid(row=0, column=3, padx=5, pady=5)
        self.end_date_entry = ttk.Entry(control_frame, width=20)
        self.end_date_entry.grid(row=0, column=4, padx=5, pady=5)
        
        ttk.Button(control_frame, text="Применить временной диапазон", command=self.update_plot).grid(row=0, column=5, padx=5, pady=5)
        ttk.Button(control_frame, text="Сбросить вид", command=self.reset_view).grid(row=0, column=6, padx=5, pady=5)
        
        # Конфигурация серий
        self.series_frame = ttk.LabelFrame(control_frame, text="Настройка серий")
        self.series_frame.grid(row=1, column=0, columnspan=7, sticky="ew", padx=5, pady=5)
        
        # Область построения графика
        self.figure, self.ax = plt.subplots(figsize=(12, 8), facecolor='black')
        self.canvas = FigureCanvasTkAgg(self.figure, plot_frame)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # Панель инструментов
        self.toolbar = NavigationToolbar2Tk(self.canvas, plot_frame)
        self.toolbar.update()
        
        # Стилизация графика
        self.figure.patch.set_facecolor('#333333')
        self.ax.set_facecolor('black')
        self.ax.grid(True, color='#444444')
        self.ax.tick_params(colors='white')
        for spine in self.ax.spines.values():
            spine.set_color('#444444')
            
    def load_data(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("CSV файлы", "*.csv"), ("Excel файлы", "*.xlsx;*.xls")]
        )
        
        if not file_path:
            return
            
        # Загрузка данных в зависимости от расширения файла
        if file_path.endswith(('.xlsx', '.xls')):
            self.data = pd.read_excel(file_path)
        else:
            self.data = pd.read_csv(file_path)
            
        # Очистка существующей конфигурации серий
        for widget in self.series_frame.winfo_children():
            widget.destroy()
            
        # Получение имен столбцов
        self.columns = list(self.data.columns)
        
        # Создание интерфейса настройки серий
        ttk.Label(self.series_frame, text="Параметр").grid(row=0, column=0, padx=5, pady=5)
        ttk.Label(self.series_frame, text="Столбец времени").grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(self.series_frame, text="Столбец значений").grid(row=0, column=2, padx=5, pady=5)
        ttk.Label(self.series_frame, text="Цвет").grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(self.series_frame, text="Отображать").grid(row=0, column=4, padx=5, pady=5)
        ttk.Label(self.series_frame, text="Мин. значение").grid(row=0, column=5, padx=5, pady=5)
        ttk.Label(self.series_frame, text="Макс. значение").grid(row=0, column=6, padx=5, pady=5)
        
        # Создание интерфейса для максимум 6 серий (можно расширить)
        self.series_controls = []
        for i in range(6):
            series_name = ttk.Entry(self.series_frame, width=15)
            series_name.grid(row=i+1, column=0, padx=5, pady=2)
            series_name.insert(0, f"Серия {i+1}")
            
            time_col = ttk.Combobox(self.series_frame, values=self.columns, width=15)
            time_col.grid(row=i+1, column=1, padx=5, pady=2)
            
            value_col = ttk.Combobox(self.series_frame, values=self.columns, width=15)
            value_col.grid(row=i+1, column=2, padx=5, pady=2)
            
            color_var = tk.StringVar(value=self.colors[i])
            color_entry = ttk.Entry(self.series_frame, width=8, textvariable=color_var)
            color_entry.grid(row=i+1, column=3, padx=5, pady=2)
            
            show_var = tk.BooleanVar(value=True)
            show_check = ttk.Checkbutton(self.series_frame, variable=show_var)
            show_check.grid(row=i+1, column=4, padx=5, pady=2)
            
            min_val = ttk.Entry(self.series_frame, width=8)
            min_val.grid(row=i+1, column=5, padx=5, pady=2)
            
            max_val = ttk.Entry(self.series_frame, width=8)
            max_val.grid(row=i+1, column=6, padx=5, pady=2)
            
            # Попытка автоматически выбрать столбцы с датой и временем
            for col in self.columns:
                if 'дат' in col.lower() or 'врем' in col.lower() or 'date' in col.lower() or 'time' in col.lower():
                    time_col.set(col)
                    break
                    
            self.series_controls.append({
                'name': series_name,
                'time_col': time_col,
                'value_col': value_col,
                'color': color_var,
                'show': show_var,
                'min_val': min_val,
                'max_val': max_val
            })
            
        ttk.Button(self.series_frame, text="Обновить график", command=self.update_plot).grid(
            row=7, column=0, columnspan=7, padx=5, pady=10)
    
    def update_plot(self):
        if self.data is None:
            return
            
        # Очистка графика
        self.ax.clear()
        self.axes = [self.ax]  # Основная ось
        
        # Установка временного диапазона, если указан
        time_range_start = None
        time_range_end = None
        
        if self.start_date_entry.get():
            try:
                time_range_start = pd.to_datetime(self.start_date_entry.get())
            except:
                print("Неверный формат начальной даты. Используйте ГГГГ-ММ-ДД ЧЧ:ММ:СС")
                
        if self.end_date_entry.get():
            try:
                time_range_end = pd.to_datetime(self.end_date_entry.get())
            except:
                print("Неверный формат конечной даты. Используйте ГГГГ-ММ-ДД ЧЧ:ММ:СС")
        
        # Построение каждой серии
        legend_elements = []
        
        for i, series in enumerate(self.series_controls):
            # Пропуск, если не выбраны столбцы или не отображается
            if not series['time_col'].get() or not series['value_col'].get() or not series['show'].get():
                continue
                
            # Получение столбцов
            time_col = series['time_col'].get()
            value_col = series['value_col'].get()
            
            # Убедимся, что столбец времени имеет формат datetime
            try:
                time_data = pd.to_datetime(self.data[time_col])
            except:
                print(f"Ошибка преобразования {time_col} в формат datetime. Пропуск серии.")
                continue
                
            # Получение значений
            try:
                value_data = self.data[value_col].astype(float)
            except:
                print(f"Ошибка преобразования {value_col} в числовой формат. Пропуск серии.")
                continue
                
            # Создание временного DataFrame с временем и значением
            temp_df = pd.DataFrame({
                'time': time_data,
                'value': value_data
            })
            
            # Применение фильтрации по времени, если указана
            if time_range_start:
                temp_df = temp_df[temp_df['time'] >= time_range_start]
            if time_range_end:
                temp_df = temp_df[temp_df['time'] <= time_range_end]
                
            # Пропуск, если нет данных
            if len(temp_df) == 0:
                continue
                
            color = series['color'].get()
            
            # Создание вторичной оси Y для всех графиков, кроме первого
            if i > 0:
                ax2 = self.ax.twinx()
                # Смещение правых осей
                ax2.spines['right'].set_position(('outward', 60 * (i-1)))
                self.axes.append(ax2)
                current_ax = ax2
            else:
                current_ax = self.ax
                
            # Установка цвета оси
            current_ax.tick_params(axis='y', colors=color)
            current_ax.spines['right' if i > 0 else 'left'].set_color(color)
            
            # Построение данных
            line, = current_ax.plot(temp_df['time'], temp_df['value'], color=color, linewidth=1.5)
            
            # Установка пределов по оси Y, если указаны
            min_val = series['min_val'].get()
            max_val = series['max_val'].get()
            
            if min_val and max_val:
                try:
                    current_ax.set_ylim(float(min_val), float(max_val))
                except:
                    pass
                    
            # Добавление в легенду
            legend_elements.append(line)
            
        # Форматирование
        self.ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
        self.ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=10))
        plt.setp(self.ax.get_xticklabels(), rotation=45, ha='right')
        
        # Настройка сетки
        self.ax.grid(True, color='#444444')
        self.ax.set_facecolor('black')
        
        # Легенда
        names = [s['name'].get() for s in self.series_controls if 
                 s['time_col'].get() and s['value_col'].get() and s['show'].get()]
        
        if legend_elements:
            self.ax.legend(legend_elements, names, loc='upper left', 
                          facecolor='#333333', edgecolor='#444444', labelcolor='white')
        
        # Отображение графика
        self.canvas.draw()
    
    def reset_view(self):
        # Очистка полей ввода временного диапазона и сброс графика
        self.start_date_entry.delete(0, tk.END)
        self.end_date_entry.delete(0, tk.END)
        self.update_plot()

# Запуск приложения
if __name__ == "__main__":
    root = tk.Tk()
    app = MultiAxisPlotter(root)
    root.mainloop()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.widgets import Button, TextBox
import numpy as np
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import filedialog, ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

class MultiParameterPlotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Визуализация нескольких параметров")
        self.root.geometry("1200x800")
        
        # Переменные для хранения данных
        self.df = None
        self.params = []
        self.datetime_column = None
        self.colors = ['red', 'green', 'white', 'cyan', 'magenta', 'yellow']
        
        # Создание фрейма для загрузки данных
        self.data_frame = ttk.LabelFrame(root, text="Управление данными")
        self.data_frame.pack(fill="x", padx=10, pady=5)
        
        # Кнопка загрузки данных
        self.load_button = ttk.Button(self.data_frame, text="Загрузить файл", command=self.load_data)
        self.load_button.pack(side="left", padx=5, pady=5)
        
        # Создание фрейма для временного диапазона
        self.time_frame = ttk.LabelFrame(root, text="Временной диапазон")
        self.time_frame.pack(fill="x", padx=10, pady=5)
        
        # Поля для ввода временного диапазона
        ttk.Label(self.time_frame, text="Начало:").grid(row=0, column=0, padx=5, pady=5)
        self.start_date_entry = ttk.Entry(self.time_frame, width=20)
        self.start_date_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.time_frame, text="Конец:").grid(row=0, column=2, padx=5, pady=5)
        self.end_date_entry = ttk.Entry(self.time_frame, width=20)
        self.end_date_entry.grid(row=0, column=3, padx=5, pady=5)
        
        # Кнопка применения временного диапазона
        self.apply_time_button = ttk.Button(self.time_frame, text="Применить", command=self.update_time_range)
        self.apply_time_button.grid(row=0, column=4, padx=5, pady=5)
        
        # Кнопка сброса временного диапазона
        self.reset_time_button = ttk.Button(self.time_frame, text="Сбросить", command=self.reset_time_range)
        self.reset_time_button.grid(row=0, column=5, padx=5, pady=5)
        
        # Предустановленные временные диапазоны
        self.time_presets_frame = ttk.Frame(self.time_frame)
        self.time_presets_frame.grid(row=1, column=0, columnspan=6, padx=5, pady=5)
        
        ttk.Button(self.time_presets_frame, text="Последний час", 
                  command=lambda: self.set_time_preset(hours=1)).pack(side="left", padx=5)
        ttk.Button(self.time_presets_frame, text="Последние 6 часов", 
                  command=lambda: self.set_time_preset(hours=6)).pack(side="left", padx=5)
        ttk.Button(self.time_presets_frame, text="Последние 24 часа", 
                  command=lambda: self.set_time_preset(hours=24)).pack(side="left", padx=5)
        ttk.Button(self.time_presets_frame, text="Последние 7 дней", 
                  command=lambda: self.set_time_preset(days=7)).pack(side="left", padx=5)
        ttk.Button(self.time_presets_frame, text="Последний месяц", 
                  command=lambda: self.set_time_preset(days=30)).pack(side="left", padx=5)
        
        # Создание фрейма для графика
        self.plot_frame = ttk.Frame(root)
        self.plot_frame.pack(fill="both", expand=True, padx=10, pady=5)
        
        # Создание области для отображения информации о параметрах
        self.info_frame = ttk.LabelFrame(root, text="Информация о параметрах")
        self.info_frame.pack(fill="x", padx=10, pady=5)
        
        # Установка начальных значений
        self.fig = None
        self.canvas = None
        self.axes = []
        self.lines = []
        self.param_labels = []
        
        # Инициализация пустого графика
        self.init_plot()
        
    def init_plot(self):
        """Инициализация пустого графика"""
        if self.canvas:
            for widget in self.plot_frame.winfo_children():
                widget.destroy()
        
        self.fig, self.ax1 = plt.subplots(figsize=(12, 6), facecolor='black')
        self.ax1.set_facecolor('black')
        self.ax1.grid(color='gray', linestyle='-', linewidth=0.5, alpha=0.3)
        
        # Настройка осей и меток
        self.ax1.tick_params(axis='x', colors='white')
        self.ax1.tick_params(axis='y', colors='white')
        self.ax1.spines['bottom'].set_color('white')
        self.ax1.spines['top'].set_color('white')
        self.ax1.spines['left'].set_color('white')
        self.ax1.spines['right'].set_color('white')
        
        self.axes = [self.ax1]
        self.lines = []
        
        # Создание холста Matplotlib
        self.canvas = FigureCanvasTkAgg(self.fig, self.plot_frame)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        # Добавление панели инструментов
        self.toolbar = NavigationToolbar2Tk(self.canvas, self.plot_frame)
        self.toolbar.update()
        self.toolbar.pack(side=tk.BOTTOM, fill=tk.X)
        
    def load_data(self):
        """Загрузка данных из файла"""
        file_path = filedialog.askopenfilename(
            filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx;*.xls")]
        )
        
        if not file_path:
            return
        
        try:
            if file_path.endswith(('.xlsx', '.xls')):
                self.df = pd.read_excel(file_path)
            else:
                self.df = pd.read_csv(file_path)
            
            # Открываем окно выбора столбцов
            self.select_columns()
            
        except Exception as e:
            tk.messagebox.showerror("Ошибка загрузки", f"Ошибка при загрузке файла: {str(e)}")
    
    def select_columns(self):
        """Открытие окна для выбора столбцов для отображения"""
        select_window = tk.Toplevel(self.root)
        select_window.title("Выбор столбцов для отображения")
        select_window.geometry("400x400")
        
        # Список столбцов
        ttk.Label(select_window, text="Выберите столбец с датой и временем:").pack(pady=5)
        
        datetime_var = tk.StringVar()
        datetime_combo = ttk.Combobox(select_window, textvariable=datetime_var, values=list(self.df.columns))
        datetime_combo.pack(pady=5, padx=10, fill="x")
        
        # Определение столбца с датой/временем автоматически
        for col in self.df.columns:
            if 'date' in col.lower() or 'time' in col.lower() or 'datetime' in col.lower():
                datetime_combo.set(col)
                break
        
        ttk.Label(select_window, text="Выберите параметры для отображения:").pack(pady=5)
        
        # Создаем чекбоксы для каждого столбца (кроме datetime)
        param_vars = {}
        param_colors = {}
        param_frame = ttk.Frame(select_window)
        param_frame.pack(fill="both", expand=True, padx=10, pady=5)
        
        row = 0
        for i, col in enumerate(self.df.columns):
            var = tk.BooleanVar(value=False)
            param_vars[col] = var
            
            frame = ttk.Frame(param_frame)
            frame.grid(row=row, column=0, sticky="w", pady=2)
            
            ttk.Checkbutton(frame, text=col, variable=var).pack(side="left")
            
            color_var = tk.StringVar(value=self.colors[i % len(self.colors)])
            param_colors[col] = color_var
            
            color_label = ttk.Label(frame, text="Цвет:")
            color_label.pack(side="left", padx=(20, 5))
            
            color_combo = ttk.Combobox(frame, textvariable=color_var, values=self.colors, width=10)
            color_combo.pack(side="left")
            
            row += 1
        
        # Кнопка подтверждения
        ttk.Button(select_window, text="Применить", 
                  command=lambda: self.apply_selection(datetime_var.get(), param_vars, param_colors, select_window)).pack(pady=10)
    
    def apply_selection(self, datetime_column, param_vars, param_colors, window):
        """Применение выбранных столбцов и отображение графика"""
        self.datetime_column = datetime_column
        
        # Преобразуем столбец с датой/временем
        try:
            self.df[self.datetime_column] = pd.to_datetime(self.df[self.datetime_column])
        except Exception as e:
            tk.messagebox.showerror("Ошибка преобразования", 
                                  f"Ошибка при преобразовании столбца даты/времени: {str(e)}")
            return
        
        # Получаем выбранные параметры
        self.params = []
        self.param_colors = {}
        
        for col, var in param_vars.items():
            if var.get() and col != self.datetime_column:
                self.params.append(col)
                self.param_colors[col] = param_colors[col].get()
        
        if not self.params:
            tk.messagebox.showwarning("Предупреждение", "Не выбрано ни одного параметра для отображения")
            return
        
        window.destroy()
        
        # Устанавливаем начальный временной диапазон
        min_date = self.df[self.datetime_column].min()
        max_date = self.df[self.datetime_column].max()
        
        self.start_date_entry.delete(0, tk.END)
        self.start_date_entry.insert(0, min_date.strftime("%Y-%m-%d %H:%M:%S"))
        
        self.end_date_entry.delete(0, tk.END)
        self.end_date_entry.insert(0, max_date.strftime("%Y-%m-%d %H:%M:%S"))
        
        # Обновляем график
        self.update_plot()
        
    def update_plot(self):
        """Обновление графика с выбранными параметрами"""
        if self.df is None or not self.params:
            return
            
        # Очищаем предыдущий график
        for widget in self.plot_frame.winfo_children():
            widget.destroy()
            
        for widget in self.info_frame.winfo_children():
            widget.destroy()
        
        # Создаем новый график
        self.fig, self.ax1 = plt.subplots(figsize=(12, 6), facecolor='black')
        self.ax1.set_facecolor('black')
        self.axes = [self.ax1]
        self.lines = []
        
        # Настройка основной оси
        self.ax1.tick_params(axis='x', colors='white')
        self.ax1.tick_params(axis='y', colors='white')
        self.ax1.grid(color='gray', linestyle='-', linewidth=0.5, alpha=0.3)
        
        # Получаем временной диапазон
        try:
            start_date = pd.to_datetime(self.start_date_entry.get())
            end_date = pd.to_datetime(self.end_date_entry.get())
            
            # Фильтруем данные по временному диапазону
            mask = (self.df[self.datetime_column] >= start_date) & (self.df[self.datetime_column] <= end_date)
            filtered_df = self.df[mask]
            
            if filtered_df.empty:
                tk.messagebox.showwarning("Предупреждение", "Нет данных в выбранном диапазоне")
                return
                
        except Exception as e:
            tk.messagebox.showerror("Ошибка", f"Ошибка при анализе диапазона дат: {str(e)}")
            return
        
        # Отображаем каждый параметр
        for i, param in enumerate(self.params):
            if i == 0:
                ax = self.ax1
                ax.set_ylabel(param, color=self.param_colors[param])
            else:
                # Создаем новую ось Y для каждого дополнительного параметра
                ax = self.ax1.twinx()
                
                # Настраиваем позицию оси
                ax.spines['right'].set_position(('outward', 60 * (i-1)))
                ax.set_ylabel(param, color=self.param_colors[param])
                
                self.axes.append(ax)
            
            # Отрисовываем линию
            line, = ax.plot(filtered_df[self.datetime_column], filtered_df[param], 
                          color=self.param_colors[param], linewidth=1.5)
            self.lines.append(line)
            
            # Настройка цвета оси и делений
            ax.tick_params(axis='y', colors=self.param_colors[param])
            ax.spines['right'].set_color(self.param_colors[param])
            
            # Добавляем информацию о текущем значении параметра
            frame = ttk.Frame(self.info_frame)
            frame.pack(side="left", padx=10, pady=5)
            
            ttk.Label(frame, text=f"{param}:", foreground=self.param_colors[param]).pack(side="left")
            
            last_value = filtered_df[param].iloc[-1] if not filtered_df.empty else "Н/Д"
            value_label = ttk.Label(frame, text=str(last_value))
            value_label.pack(side="left", padx=5)
        
        # Настройка форматирования оси X (дата)
        self.ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S\n%d.%m.%y'))
        plt.setp(self.ax1.get_xticklabels(), rotation=0)
        
        # Настройка заголовка
        self.ax1.set_title("Мультипараметрический график", color='white')
        
        # Создание холста Matplotlib
        self.canvas = FigureCanvasTkAgg(self.fig, self.plot_frame)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        # Добавление панели инструментов
        self.toolbar = NavigationToolbar2Tk(self.canvas, self.plot_frame)
        self.toolbar.update()
        self.toolbar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Автоматически подстраиваем компоновку
        self.fig.tight_layout()
    
    def update_time_range(self):
        """Обновление временного диапазона"""
        self.update_plot()
    
    def reset_time_range(self):
        """Сброс временного диапазона к полному"""
        if self.df is not None and self.datetime_column is not None:
            min_date = self.df[self.datetime_column].min()
            max_date = self.df[self.datetime_column].max()
            
            self.start_date_entry.delete(0, tk.END)
            self.start_date_entry.insert(0, min_date.strftime("%Y-%m-%d %H:%M:%S"))
            
            self.end_date_entry.delete(0, tk.END)
            self.end_date_entry.insert(0, max_date.strftime("%Y-%m-%d %H:%M:%S"))
            
            self.update_plot()
    
    def set_time_preset(self, hours=None, days=None):
        """Установка предустановленного временного диапазона"""
        if self.df is None or self.datetime_column is None:
            return
            
        max_date = self.df[self.datetime_column].max()
        
        if hours:
            min_date = max_date - timedelta(hours=hours)
        elif days:
            min_date = max_date - timedelta(days=days)
        else:
            return
            
        self.start_date_entry.delete(0, tk.END)
        self.start_date_entry.insert(0, min_date.strftime("%Y-%m-%d %H:%M:%S"))
        
        self.end_date_entry.delete(0, tk.END)
        self.end_date_entry.insert(0, max_date.strftime("%Y-%m-%d %H:%M:%S"))
        
        self.update_plot()

# Запуск приложения
if __name__ == "__main__":
    root = tk.Tk()
    app = MultiParameterPlotApp(root)
    root.mainloop()